In [1]:
# %load_ext autoreload
# %autoreload 1
# %aimport foo
import os
import sys
from packages.models import Model
from packages.helper import traj_speed, play_trajs, beta
import numpy as np
from numpy import gradient
from numpy.linalg import norm
import pickle
import matplotlib.pyplot as plt
from packages.tester import Tester
from packages.simulator import Agent
from packages import data_container
from mpl_toolkits.mplot3d import Axes3D
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container
%matplotlib qt


# Steering to goal

In [149]:
file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Fajen_steer_exp1a_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

In [7]:
'''Animate data'''
%matplotlib qt
i = 51
# p_obst = np.array(data.info['p_obst'][i])
t0 = data.info['goal_onset'][i]
p_goal = np.array(data.info['p_goal'][i])
p_subj = np.array(data.get_traj(i))[:,:2]
trajs = [p_goal, p_subj]
ws = [data.info['w_goal'], 0.4]
play_trajs(trajs, ws, data.Hz)

In [119]:
'''Plot speed'''
%matplotlib qt
Hz = data.Hz
for i in range(len(data.trajs)):
    subj = np.array(data.get_traj(i))[:, :2]
    if data.info['subj_id'][i] == 5:
        for j in range(1, len(subj)):
            spd1 = norm([(a - b) * Hz for a, b in zip(subj[j], subj[j-1])])
            spd2 = norm([(a - b) * Hz for a, b in zip(subj[j+1], subj[j])])
            if spd1 > 0.22 and spd2 > spd1:
                t0 = j
                break
        t1 = t0 + 3 * Hz
        spd = traj_speed(subj[t0:t1], data.Hz)
        plt.plot(spd)

In [178]:
'''Steer2Goal test'''

tests = []

w = 0.4
mass_spring_approach1 = {'name': 'mass_spring_approach1', 
                         'b_1': 3.25, 'b_2': 4.8, 'k_1':7.5, 'k_2': 6, 'c_1': 0.4, 'c_2': 0.4}

models = {'approach': mass_spring_approach1}
agent = Agent(1, goal_id=0, w=w, models=models)
info = {'w': {'goals': [0.1]}, 'ps':[]}
goals = []
subjs = []
obsts = []
Hz = data.Hz
for i in range(len(data.trajs)):
    if i in data.dump:
        continue

    goal = data.info['p_goal'][i]
    subj = np.array(data.get_traj(i))[:,0:2].tolist()
    # Define the start and end of a trial for fitting
    for j in range(1, len(subj)):
        spd1 = norm([(a - b) * Hz for a, b in zip(subj[j], subj[j-1])])
        spd2 = norm([(a - b) * Hz for a, b in zip(subj[j+1], subj[j])])
        if spd1 > 0.22 and spd2 > spd1:
            t0 = j
            break
    t1 = t0 + 3 * Hz
    goals.append([goal[t0:t1]])
    subjs.append(subj[t0:t1])
    info['ps'].append(data.info['ps'][i])


tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
tests[-1].simulate()

#===========================================


In [179]:
'''Test metrics'''
i_test = 0
s_RMSE = tests[i_test].test('s_RMSE')
print(np.mean(tests[i_test].test('p_dist')),       
      np.mean(s_RMSE),
      np.mean(tests[i_test].test('phi_RMSE')),
      np.mean(tests[i_test].test('s_MAE')),
      np.mean(tests[i_test].test('phi_MAE')))

0.15695017477627626 0.04752809234726686 0.13648189685251022 0.039232094326499996 0.11374536152592071


In [189]:
'''Animate trial'''
%matplotlib qt
i_test = 0
i = 10
goal = tests[i_test].goals[i][0]
# obst = tests[i_test].obsts[i][0]
subjs = tests[i_test].subjs[i]
pred = tests[i_test].p_pred[i]
play_trajs([goal, subjs, pred], [None, 0.4, 0.4], tests[i_test].Hz, labels=('goal','subj','model'))
print(tests[i_test].test('p_dist', i_trial=i),       
      tests[i_test].test('s_RMSE', i_trial=i),
      tests[i_test].test('phi_RMSE', i_trial=i),
      tests[i_test].test('s_MAE', i_trial=i),
      tests[i_test].test('phi_MAE', i_trial=i))


[0.1441908578326049] [0.024881187922919174] [0.12065784952640389] [0.01968562919926966] [0.10658817624657614]


In [122]:
'''Find worst trials'''
i = np.argmax(s_RMSE)
# print(data.info['subj_id'][i], i, s_RMSE[i])
for i in range(len(s_RMSE)):
    if 0.1475 - s_RMSE[i] < 0.01:
        print(data.info['subj_id'][i], i, s_RMSE[i])

0 32 0.14033975714644034
0 42 0.13798210321780272
1 99 0.14718993263943536
5 402 0.1387086725540348
5 432 0.14757106561083363


In [166]:
'''Loading fitting results'''
b2 = []
k2 = []
s_RMSE = []
s_MAE = []
s_MSE = []

with open('fitting_results_Fajen_steer_exp1a_2020-6-7-1-40-35.csv', 'r') as file:
    for line in file:
        if line != '\n':
            line = line.split(',')
            args = line[2].split('/')
#             if float(args[1]) < 5 and float(args[3]) < 5:
            b2.append(float(args[1]))
            k2.append(float(args[3]))
            err = line[4].split('/')        
            s_RMSE.append(float(err[0]))
            s_MAE.append(float(err[1]))
            s_MSE.append(float(err[2]))

In [167]:
'''Plot fitting results'''
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(b2, k2, s_RMSE, marker='o', c='r')
ax.set_xlabel('b2')
ax.set_ylabel('k2')
ax.set_zlabel('s')

Text(0.5, 0, 's')

In [177]:
'''Find the best fit parameters'''
i0 = np.argmin(s_RMSE)
print(b2[i0], k2[i0], s_RMSE[i0])
for i in range(len(b2)):
    if s_RMSE[i] - s_RMSE[i0] < 0.00005:
        print(b2[i], k2[i], s_RMSE[i])
        plt.scatter(k2[i], b2[i])
plt.plot([3.07, 9.98], [2.84, 7.58])

4.83 6.04 0.0475
4.83 6.04 0.0475
4.43 5.44 0.0475
4.63 5.64 0.0475


# movObst

In [3]:
file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data\\Cohen_movObst_exp1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

In [ ]:
'''Animate data'''
%matplotlib qt
i = 0
p_obst = np.array(data.info['p_obst'][i])
p_goal = np.array(data.info['p_goal'][i])
p_subj = np.array(data.get_traj(i))[:,:2]
trajs = [p_goal, p_obst, p_subj]
ws = [data.info['w_goal'], data.info['w_obst'], 0.4]
play_trajs(trajs, ws, data.Hz)

In [17]:
'''Plot speed'''
for i in range(len(data.trajs)):
    if i not in data.dump:
        t0 = data.info['stimuli_onset'][i]
        spd = traj_speed(np.array(data.get_traj(i))[t0:,:2], data.Hz)
        plt.plot(spd)

In [39]:
'''movObst test'''

tests = []

w = 0.4
vector_approach = {'name': 'vector_approach', 'ps': 1.3, 't_relax': 1}
perpendicular_acceleration_avoid = {'name': 'perpendicular_acceleration_avoid', 'k': 1, 'c': 1}

mass_spring_approach1 = {'name': 'mass_spring_approach1', 
                         'ps': 1.3, 'b_1': 3.25, 'b_2': 2.3, 'k_1':7.5, 'k_2': 2, 'c_1': 0.4, 'c_2': 0.4}
cohen_avoid3 = {'name': 'cohen_avoid3', 'k_1': 530, 'k_2': 50, 'c_5': 6, 'c_6': 1.3, 'c_7': 6, 'c_8': 1.3}

models = {'approach': mass_spring_approach1, 'avoid': cohen_avoid3}
agent = Agent(1, goal_id=0, w=w, models=models)
info = {'w': {'goals': [0.1], 'obsts': [0.1]}, 'ps': []}
goals = []
obsts = []
subjs = []
for i in range(len(data.trajs)):
    if i in data.dump:
        continue

    goal = data.info['p_goal'][i]
    obst = data.info['p_obst'][i]
    subj = np.array(data.get_traj(i))[:,0:2].tolist()
    # Define the start and end of a trial for fitting
    t0 = data.info['stimuli_onset'][i]
    for j in range(t0, len(subj)):
        p0 = subj[j]
        p1 = obst[j]
        v0 = [(a - b) * data.Hz for a, b in zip(subj[j+1], subj[j])]
        if abs(beta(p0, p1, v0)) > np.pi / 2:
            t1 = j
            break    
    goals.append([goal[t0:t1]])
    obsts.append([obst[t0:t1]])
    subjs.append(subj[t0:t1])
    info['ps'].append(data.info['ps'][i])

Hz = data.Hz
tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
tests[-1].simulate()

#===========================================

mass_spring_approach1 = {'name': 'mass_spring_approach1', 
                         'ps': 1.3, 'b_1': 3.25, 'b_2': 2.3, 'k_1':7.5, 'k_2': 2, 'c_1': 0.4, 'c_2': 0.4}
cohen_avoid3 = {'name': 'cohen_avoid3', 'k_1': 530, 'k_2': 10, 'c_5': 6, 'c_6': 1.3, 'c_7': 1, 'c_8': 1.3}
models = {'approach': mass_spring_approach1, 'avoid': cohen_avoid3}
agent = Agent(1, goal_id=0, w=w, models=models)
tests.append(Tester(agent, goals, obsts, subjs, info, Hz))
tests[-1].simulate()

In [ ]:
'''Test metrics'''
i_test = 0
s_RMSE = tests[i_test].test('s_RMSE')
print(np.mean(tests[i_test].test('p_dist')),       
      np.mean(s_RMSE),
      np.mean(tests[i_test].test('phi_RMSE')),
      np.mean(tests[i_test].test('s_MAE')),
      np.mean(tests[i_test].test('phi_MAE')))

In [ ]:
'''Animate trial'''
%matplotlib qt
i_test = 0
i = 10
goal = tests[i_test].goals[i][0]
# obst = tests[i_test].obsts[i][0]
subjs = tests[i_test].subjs[i]
pred = tests[i_test].p_pred[i]
play_trajs([goal, subjs, pred], [None, 0.4, 0.4], tests[i_test].Hz, labels=('goal','subj','model'))
print(tests[i_test].test('p_dist', i_trial=i),       
      tests[i_test].test('s_RMSE', i_trial=i),
      tests[i_test].test('phi_RMSE', i_trial=i),
      tests[i_test].test('s_MAE', i_trial=i),
      tests[i_test].test('phi_MAE', i_trial=i))

In [ ]:
'''Loading fitting results'''
b2 = []
k2 = []
s_RMSE = []
s_MAE = []
s_MSE = []

with open('fitting_results_Fajen_steer_exp1a_2020-6-7-1-40-35.csv', 'r') as file:
    for line in file:
        if line != '\n':
            line = line.split(',')
            args = line[2].split('/')
#             if float(args[1]) < 5 and float(args[3]) < 5:
            b2.append(float(args[1]))
            k2.append(float(args[3]))
            err = line[4].split('/')        
            s_RMSE.append(float(err[0]))
            s_MAE.append(float(err[1]))
            s_MSE.append(float(err[2]))

In [ ]:
'''Plot fitting results'''
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(b2, k2, s_RMSE, marker='o', c='r')
ax.set_xlabel('b2')
ax.set_ylabel('k2')
ax.set_zlabel('s')

In [ ]:
'''Find the best fit parameters'''
i0 = np.argmin(s_RMSE)
print(b2[i0], k2[i0], s_RMSE[i0])
for i in range(len(b2)):
    if s_RMSE[i] - s_RMSE[i0] < 0.00005:
        print(b2[i], k2[i], s_RMSE[i])
        plt.scatter(k2[i], b2[i])
plt.plot([3.07, 9.98], [2.84, 7.58])